# Transformers by HuggingFace

In [ ]:
import torch
import pandas as pd

In [ ]:
protein_descriptors = pd.read_csv(r'C:\Users\gonca\Documents\GitHub\SIB_Work/protein_descriptors.csv')
protein_descriptors

,Target_ID,Target,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,O00141,MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNS...,431.0,6.168,0.000126,2211,3416,574,640,15,...,1.266,58.861,136.076,204.430,271.519,0.730,83.212,155.474,229.197,314.599
1,O14920,MSWSPSLTTQTCGAWEMKERLGTGGFGNVIRWHNQETGEQIAIKQC...,756.0,-14.843,-0.000171,3733,5900,1008,1224,40,...,0.881,79.736,151.982,245.815,333.040,0.410,75.000,148.361,221.311,308.197
2,O15111,MERPPGLRPGAGGPWEMRERLGTGGFGNVCLYQHRELDLKIAIKSC...,745.0,-8.616,-0.000102,3714,5853,985,1149,45,...,1.606,79.920,153.414,230.120,298.795,0.394,70.079,134.252,212.992,292.520
3,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,1210.0,-12.897,-0.000096,5826,9137,1597,1835,85,...,0.671,69.799,140.268,210.291,270.694,0.260,76.042,153.906,222.135,314.583
4,P04626,MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDML...,1255.0,-32.426,-0.000235,6004,9333,1633,1880,82,...,0.795,61.630,127.435,205.368,249.304,0.252,74.559,149.874,216.877,316.121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,P42345,MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQH...,2549.0,-7.013,-0.000024,12731,20074,3428,3862,130,...,0.339,68.510,144.018,212.528,287.472,0.116,76.505,138.889,222.106,295.023
225,Q9BUB5,MVSSQKLEKPIEMGSSEPLPIADGDRRRKKKRRGRATDSLPGKFED...,465.0,-3.844,-0.000075,2227,3519,613,716,22,...,1.676,74.860,126.816,193.855,259.218,0.758,93.182,181.061,248.485,352.273
226,P49759,MRHSKRTYCPDWDDKDWDYGKWRSSSSHKRRKRSHSSAQENKRCKY...,484.0,16.713,0.000292,2503,3859,747,750,22,...,1.685,58.989,120.787,193.258,271.348,0.775,151.938,215.504,293.023,375.194
227,Q9UBF8,MGDTVVEPAPLKPTSEPTSGPPGNNGGSLLSVITEGVGELSVIDPE...,816.0,-14.527,-0.000159,3983,6286,1086,1277,32,...,0.678,66.441,126.102,200.678,275.932,0.394,81.890,180.709,257.087,321.260


In [ ]:
from transformers import AutoModel, AutoTokenizer

ProtBert

In [ ]:
model_name = "Rostlab/prot_bert_bfd"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

c:\Users\gonca\anaconda3\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
protein_sequences = protein_descriptors['Target'].tolist()
tokenized_sequences = tokenizer(protein_sequences, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
with torch.no_grad():
    outputs = model(**tokenized_sequences)
    embeddings = outputs.last_hidden_state

In [ ]:
embeddings.shape

torch.Size([229, 3, 1024])

In [ ]:
import tensorflow as tf
import numpy as np

tensor_shape = embeddings.shape
num_slices, num_rows, num_cols = tensor_shape[0], tensor_shape[1], tensor_shape[2]

# Reshape the tensor to a 2D array
flat_tensor = tf.reshape(embeddings, (num_slices * num_rows, num_cols))

# Convert the tensor to a NumPy array
numpy_array = flat_tensor.numpy()

# Specify the file path for the CSV file
csv_file_path = 'transf_emb.csv'

# Save the NumPy array to a CSV file
np.savetxt(csv_file_path, numpy_array, delimiter=',')

ESM

In [ ]:
import torch
import esm

# Load 34 layer model
model, alphabet = esm.pretrained.esm1_t34_670M_UR50S()
batch_converter = alphabet.get_batch_converter()

# Placeholder data (replace this with your actual protein sequences)
protein_data = protein_descriptors

# Prepare data
raw_batch = list(zip(protein_data["Target_ID"], protein_data["Target"]))
batch_labels, batch_strs, batch_tokens = batch_converter(raw_batch)

# Extract per-residue embeddings (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[34])
token_embeddings = results["representations"][34]

# Generate per-sequence embeddings via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_embeddings = []
for i, (_, seq) in enumerate(raw_batch):
    sequence_embeddings.append(token_embeddings[i, 1 : len(seq) + 1].mean(0))
